In [2]:
using PretrainedEmbeddings

using DataDeps

In [21]:
dd_name = language_files(PretrainedEmbeddings.FastText_Bin{:fr}) |> first

"FastText fr CommonCrawl Binary/cc.fr.300.bin"

In [22]:
stat

StatStruct(mode=0o100644, size=7238894263)

In [ ]:
#=
struct entry {
  std::string word;
  int64_t count;
  entry_type type;
  std::vector<int32_t> subwords;
};
        #=

In [24]:
#https://github.com/facebookresearch/fastText/blob/master/src/dictionary.cc

In [20]:
readdir(datadep"FastText fr CommonCrawl Binary")

1-element Array{String,1}:
 "cc.fr.300.bin"

In [33]:
@enum EntryType::Int8 word_type=0 label_type=1

struct Entry
    word::String
    count::Int64
    entry_type:: EntryType
    subwords::Vector{Int32}
end
Entry()=Entry("", 0, word_type, Int32[])


Entry

In [78]:
const FASTTEXT_VERSION = Int32(12); # Version 1b 
const FASTTEXT_FILEFORMAT_MAGIC_INT32 = Int32(793712314);


function load_header(fh)
	### Check Model
    @show magic = read(fh, Int32)
    @assert magic== FASTTEXT_FILEFORMAT_MAGIC_INT32
    @show version = read(fh, Int32)
    @assert version == FASTTEXT_VERSION
    println()
end

function load_args(fh)
    ## Load Args https://github.com/facebookresearch/fastText/blob/master/src/args.cc#L261
    @show args_dim = read(fh, Int32)
    @show args_ws = read(fh, Int32)
    @show args_epoch = read(fh, Int32)
    @show args_minCount = read(fh, Int32)
    @show args_neg = read(fh, Int32)
    @show args_wordNgrams = read(fh, Int32)
    @show args_loss = read(fh, Int32)
    @show args_model = read(fh, Int32)
    @show args_bucket = read(fh, Int32)
    @show args_minn = read(fh, Int32)
    @show args_maxn = read(fh, Int32)
    @show args_lrUpdateRate = read(fh, Int32)
    @show args_t = read(fh, Float64)
    println()
end

function load_dict(fh)
    ## Load model dict, https://github.com/facebookresearch/fastText/blob/master/src/dictionary.cc#L419   
    @show size_ = read(fh, Int32)
    @show nwords = read(fh, Int32)
    @show nlabels = read(fh, Int32)
    @show ntokens = read(fh, Int64)
    @show pruneidx_size_ = read(fh, Int64)
    
    println()
    words_ = map(1:size_) do ii
        e_word=readuntil(fh, '\0')[1:end-1]
        e_count=read(fh, Int64)
        e_entry_type=read(fh, EntryType)
        Entry(e_word, e_count, e_entry_type, Int32[])
    end
    @show length(words_)
    @show words_[1]
    @show words_[2]
    @show words_[3]
    @show words_[end-1]
    @show words_[end]
    println()
    @assert pruneidx_size_ < 0 
    # Avoid loading this stuff https://github.com/facebookresearch/fastText/blob/master/src/dictionary.cc#L437
    println()
	
	words_
end

function load_matrix(fh)
    ### Load Matrix
    #https://github.com/facebookresearch/fastText/blob/master/src/matrix.cc#L114
    
    @show m_ = read(fh, Int64)
    @show n_ = read(fh, Int64)
    data = read(fh, Float32, (m_, n_)) # Note `real` is a typedef for `float32`
    @show typeof(data), size(data)
	data
end

function load_fasttext_bin(filename)
	open(filename) do fh
		load_header(fh)
		load_args(fh)
		load_dict(fh)
		
		
		@show quant_input = read(fh, Bool)
		@assert !quant_input # avoid that stuff
		input_ = load_matrix(fh)
		
		@show quant_output = read(fh, Bool)
		@assert !quant_output # avoid that stuff
		output_ = load_matrix(fh)
		
        @assert(eof(fh))
	end
end


load_fasttext_bin(@datadep_str dd_name)


magic = read(fh, Int32) = 793712314
version = read(fh, Int32) = 12

args_dim = read(fh, Int32) = 300
args_ws = read(fh, Int32) = 5
args_epoch = read(fh, Int32) = 1
args_minCount = read(fh, Int32) = 5
args_neg = read(fh, Int32) = 10
args_wordNgrams = read(fh, Int32) = 1
args_loss = read(fh, Int32) = 2
args_model = read(fh, Int32) = 1
args_bucket = read(fh, Int32) = 2000000
args_minn = read(fh, Int32) = 5
args_maxn = read(fh, Int32) = 5
args_lrUpdateRate = read(fh, Int32) = 100
args_t = read(fh, Float64) = 9.999999747378752e-6

size_ = read(fh, Int32) = 2000000
nwords = read(fh, Int32) = 2000000
nlabels = read(fh, Int32) = 0
ntokens = read(fh, Int64) = 68358270953
pruneidx_size_ = read(fh, Int64) = -1

length(words_) = 2000000
words_[1] = Entry(",", 2854010684, word_type::EntryType = 0, Int32[])
words_[2] = Entry("de", 2742946523, word_type::EntryType = 0, Int32[])
words_[3] = Entry(".", 1675680641, word_type::EntryType = 0, Int32[])
words_[end - 1] = Entry("Fautereau", 235, word_type::E

In [42]:
?readstring

search: readstring



```
readstring(stream::IO)
readstring(filename::AbstractString)
```

Read the entire contents of an I/O stream or a file as a string. The text is assumed to be encoded in UTF-8.
